In [15]:
import pickle
import cobra

In [16]:
# Load the results
with open('results.pkl', 'rb') as f:
    experiment = pickle.load(f)

In [17]:
# Load the COBRA model
alt_model = cobra.io.read_sbml_model("../../../../GEM-repos/mit1002-model/model.xml")

'' is not a valid SBML 'SId'.


In [18]:
ecoli_model = cobra.io.load_json_model("/Users/helenscott/Desktop/iJO1366.json")

In [19]:
def get_readable_metabolite_name(metabolite):
    return f"{metabolite.name} ({metabolite.id})"

In [20]:
#metabolite_id = "cpd00007_c0" # (O2)
# metabolite_id = "cpd00011_c0" # (CO2)
metabolite_id = "cpd00002_c0" # (ATP)

In [21]:
# Check if the metabolite exists in the model
if metabolite_id in alt_model.metabolites:
    metabolite = alt_model.metabolites.get_by_id(metabolite_id)
else:
    raise ValueError(f"Metabolite {metabolite_id} not found in the model.")

In [22]:
# Find all reactions containing the metabolite
reactions_with_metabolite = [reaction for reaction in alt_model.reactions if metabolite in reaction.metabolites]

In [23]:
reactions_with_metabolite

[<Reaction rxn00247_c0 at 0x284474850>,
 <Reaction rxn00225_c0 at 0x283dd6760>,
 <Reaction rxn00148_c0 at 0x283dd6910>,
 <Reaction rxn01100_c0 at 0x283df38b0>,
 <Reaction rxn01123_c0 at 0x283df3bb0>,
 <Reaction rxn00770_c0 at 0x28619ab50>,
 <Reaction rxn00175_c0 at 0x283dccd60>,
 <Reaction rxn00615_c0 at 0x283dcc880>,
 <Reaction rxn00216_c0 at 0x283cd8880>,
 <Reaction rxn00077_c0 at 0x283cd8c10>,
 <Reaction rxn01275_c0 at 0x283cd54c0>,
 <Reaction rxn08173_c0 at 0x283cd5e20>,
 <Reaction rxn00646_c0 at 0x283dcceb0>,
 <Reaction rxn01673_c0 at 0x283c7f790>,
 <Reaction rxn00107_c0 at 0x283c7fb80>,
 <Reaction rxn09240_c0 at 0x283c767c0>,
 <Reaction rxn00179_c0 at 0x283c76c40>,
 <Reaction rxn08137_c0 at 0x285e71a30>,
 <Reaction rxn03181_c0 at 0x285e46e20>,
 <Reaction rxn08017_c0 at 0x285bbca90>,
 <Reaction rxn00361_c0 at 0x285b8feb0>,
 <Reaction rxn04384_c0 at 0x285ba2700>,
 <Reaction rxn00100_c0 at 0x283c7f3d0>,
 <Reaction rxn00789_c0 at 0x283c76a90>,
 <Reaction rxn00097_c0 at 0x2846cafd0>,


In [24]:
# Get the fluxes for a specific cycle (the first, index 0) of the simulation
fluxes = experiment.fluxes_by_species[''].iloc[0]

In [25]:
# Print the nonzero flux reactions containing the metabolite, flux, and bounds
for reaction in reactions_with_metabolite:
    reaction_id = reaction.id
    flux = fluxes[reaction_id]
    
    # Filter out reactions with near-zero flux (using a threshold, e.g., 1e-6)
    if abs(flux) > 1e-6:
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction_id} | {reaction.name} | {readable_reaction_string} : {flux} | Bounds: [{lower_bound}, {upper_bound}]")

In [26]:
# Print the nonzero flux reactions containing the metabolite, flux, and bounds
for reaction in reactions_with_metabolite:
    reaction_id = reaction.id
    flux = fluxes[reaction_id]
    
    # Filter out reactions with near-zero flux (using a threshold, e.g., 1e-6)
    if abs(flux) < 1e-6:
        continue
    
    # Get reactions where CO2 is being used (i.e. the it is a product and the 
    # flux is negative or it is a reactant and the flux is positive)
    if (metabolite in reaction.products and flux > 0) or (metabolite in reaction.reactants and flux < 0):
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction_id} | {reaction.name} | {readable_reaction_string} : {flux} | Bounds: [{lower_bound}, {upper_bound}]")

In [27]:
# Get all the reactions that can produce ATP
metabolite_id = "cpd00002_c0" # (ATP)
metabolite = alt_model.metabolites.get_by_id(metabolite_id)

for reaction in alt_model.reactions:
    if metabolite in reaction.products:
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction.id} | {bigg_id} | Bounds: [{lower_bound}, {upper_bound}] | {reaction.name} | {readable_reaction_string}")
    if metabolite in reaction.reactants and reaction.lower_bound < 0:
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction.id} | {bigg_id} | Bounds: [{lower_bound}, {upper_bound}] | {reaction.name} | {readable_reaction_string}")

rxn00225_c0 | ACKr | Bounds: [-1000.0, 1000.0] | ATP:acetate phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Acetate_c0 (cpd00029_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 Acetylphosphate_c0 (cpd00196_c0)
rxn00148_c0 | ['CDC19', 'PYK', 'PYK2'] | Bounds: [-1000.0, 0.0] | ATP:pyruvate 2-O-phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Pyruvate_c0 (cpd00020_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 Phosphoenolpyruvate_c0 (cpd00061_c0) + 1.0 H+_c0 (cpd00067_c0)
rxn01100_c0 | ['PGK1', 'PGK'] | Bounds: [-1000.0, 1000.0] | ATP:3-phospho-D-glycerate 1-phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 3-Phosphoglycerate_c0 (cpd00169_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 1,3-Bisphospho-D-glycerate_c0 (cpd00203_c0)
rxn00770_c0 | No BiGG ID | Bounds: [-1000.0, 1000.0] | ATP:D-ribose-5-phosphate diphosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 ribose-5-phosphate_c0 (cpd00101_c0) + 1.0 AMP_c0 (cpd00018_c0) + 1.0 H+_c0 (cpd00067_c0) + 1.0 PRPP_c0 (cpd00103_c0)
rxn08173_c0 | [

In [28]:
for reaction in alt_model.reactions:
    if (metabolite in reaction.products) or (metabolite in reaction.reactants and reaction.lower_bound < 0):
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
            ec_reaction = ecoli_model.reactions.get_by_id(bigg_id)
            ec_lower_bound = ec_reaction.lower_bound
            ec_upper_bound = ec_reaction.upper_bound
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        # Look up the reaction in the E. coli model
        for id in b
        print(f"{reaction.id} / {bigg_id}/ {reaction.name}")
        print("----------------------------------")
        print(f"Bounds in ALT: [{lower_bound}, {upper_bound}]")
        print(f"Bounds in E. coli: [{ec_lower_bound}, {ec_upper_bound}]")
        print(f"{readable_reaction_string}\n")

SyntaxError: invalid syntax (217187064.py, line 14)